<a href="https://colab.research.google.com/github/sameepshrestha/Mission-Chatbot/blob/main/word_vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

word2vec model : https://arxiv.org/pdf/1310.4546.pdf 

used for representing the different words as  vector capturing the context on which they are used.
First things first let us get the total vocab of our data ,we still need a little cleaning left in our data lets do that first shall we.

In [136]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization 
import tqdm
from tensorflow.keras import Model
from tensorflow.keras.layers import Dot, Embedding, Flatten
%matplotlib

Using matplotlib backend: agg


In [137]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [138]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [139]:
dataset = pd.read_csv('/content/drive/MyDrive/chat bots/final_dataset.csv')
dataset.head()

,From,Message,reply
0,Puspa Rai,O ho k x hou,Thik. Xa
1,Dipesh Chapagain,Hi Science test tomarrow,hlo \ni am a friend of sameep...
2,Spponge Bob,hhahaha sale malai k taha :D talai taha theyo ...,Ah ta aja tst ma gad
3,Spponge Bob,hhahahhaa k test ma pass hunxas ta :D,Xup pas ta humxu ful marks audaina bujis
4,Spponge Bob,hhahhaha :D :D sale maile tero wall pic hali d...,Ha kasari xange hanu pas


In [140]:
import re
def remove(message):
    pure = ''
    for letter in message:
        if letter in 'abcdefghijklmnopqrstuvwxyz' or letter == ' ':
              pure = pure + letter
    # print(pure)
    return pure
        
def remove_haha(message):
    sentence=''
    for word in message.split():
        if re.search('haha',word):
            sentence = sentence+' '+ 'haha'
        else :
            sentence = sentence+' '+ word
    return sentence
def remove_and_clean(message):  
    
    clean_message = remove(message.lower())
    clean_message = remove_haha(clean_message)
    # print(clean_message)
    return clean_message 
dataset['clean_message']= dataset.apply(lambda x : remove_and_clean(x['Message']) ,axis=1)
dataset['clean_reply']= dataset.apply(lambda x : remove_and_clean(x['reply']) ,axis=1)

removing the unnecessary long messages to decrease the sequence of the model to make the padding necessary for sentence with smaller number of words be small so that the model dosent fit on the padding data

'from about 38k data 1400 empty and long message dataset were deleted

In [141]:
largest = []
vocab_size = []
vocab = []
empty = []
for i,(line1, line2) in enumerate(zip(dataset['clean_message'][0:],dataset['clean_reply'][0:])):
    len1 = len(line1.split())
    len2 = len(line2.split())

    vocab_size.append(len1)
    vocab_size.append(len2)
    if len1 == 0 or len2 == 0 or len1 >=30 or len2>=30 :
        empty.append(i)
    else:
        for word1 in line1.split():
            if word1 not in vocab:
                vocab.append(word1)
        for word2 in line2.split():
            if word2 not in vocab:
                vocab.append(word2)
print(len(vocab))
        

38533


In [142]:
len(empty)
dataset = dataset.drop(labels=None,axis=0,index=empty)
dataset.describe()

,From,Message,reply,clean_message,clean_reply
count,36152,36152,36152,36152,36152
unique,297,33708,33442,32919,32831
top,Narmada Dahal,Ah,ah,haha,ah
freq,7318,74,200,153,260


In [143]:
# dataset.describe()
dataset.head()

,From,Message,reply,clean_message,clean_reply
0,Puspa Rai,O ho k x hou,Thik. Xa,o ho k x hou,thik xa
1,Dipesh Chapagain,Hi Science test tomarrow,hlo \ni am a friend of sameep...,hi science test tomarrow,hlo i am a friend of sameep
2,Spponge Bob,hhahaha sale malai k taha :D talai taha theyo ...,Ah ta aja tst ma gad,haha sale malai k taha d talai taha theyo jas...,ah ta aja tst ma gad
3,Spponge Bob,hhahahhaa k test ma pass hunxas ta :D,Xup pas ta humxu ful marks audaina bujis,haha k test ma pass hunxas ta d,xup pas ta humxu ful marks audaina bujis
4,Spponge Bob,hhahhaha :D :D sale maile tero wall pic hali d...,Ha kasari xange hanu pas,haha d d sale maile tero wall pic hali daeko ...,ha kasari xange hanu pas


This part of the code is from the tensorflow itself. Couple of things happening in this as recquired from the paper discussed in the paper is :


*   skip grams making for the sentence given
*   next is making the context and target 
*   than for the negative sampling we recquire some words selected in randm to apply sigmoid or binary classification 
*   softmax coputionally expensive so this is selected as summing up for 40k for every step will be quite extensive

A full description can be found in tensorflow dataset 

https://www.tensorflow.org/tutorials/text/word2vec




In [144]:
all_messages = pd.concat([dataset['clean_message'],dataset['clean_reply']],axis=0)
all_messages = all_messages.reset_index(drop=True)

In [145]:
text_dataset = tf.data.Dataset.from_tensor_slices(all_messages)

In [146]:
#text vectorizer 
vocab_size = 38535
sequence_length = 30
vectorize_layer = TextVectorization(
                    split= 'whitespace',
                    output_mode='int',
                    max_tokens=vocab_size,
                    output_sequence_length=sequence_length
                    )
vectorize_layer.adapt(text_dataset.batch(1024))

In [147]:
vocab = vectorize_layer.get_vocabulary()
vocab[:20]

['',
 '[UNK]',
 'ma',
 'ho',
 'ko',
 'ni',
 'ta',
 'k',
 'xa',
 'haha',
 'la',
 'tah',
 'tyo',
 'ani',
 'ae',
 'ah',
 'hunxa',
 'chai',
 'nai',
 'ra']

In [148]:
text_vector_ds = text_dataset.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [149]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))
for seq in sequences[:5]:
  print(f"{seq} => {[vocab[i] for i in seq]}")

72304
[875   3   7 337 898   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0] => ['o', 'ho', 'k', 'x', 'hou', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
[  584   570   430 18455     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0] => ['hi', 'science', 'test', 'tomarrow', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
[    9   229    30     7  1408   202   156  1408  1843   448 27235   148
    99   135     3    12   202     0     0     0     0     0     0     0
     0     0     0     0     0     0] => ['haha', 'sale', 'malai', 'k', 'taha', 'd', 'talai', 'taha', 'theyo', 'jastai', 'lagod', 'oe', 'tero', 'sir', 'ho', 'tyo', 'd', '', '', '', '', '', '', '', '', '', '', '', '', '']
[   9    7  430    2  348 3143    6  202   

In [150]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [151]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)
print(len(targets), len(contexts), len(labels))

100%|██████████| 72304/72304 [00:47<00:00, 1517.16it/s]


407184 407184 407184


In [152]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
num_ns = 4

In [153]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: (((1024,), (1024, 5, 1)), (1024, 5)), types: ((tf.int32, tf.int64), tf.int64)>


In [154]:
class Word2Vec(Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)
    self.dots = Dot(axes=(3, 2))
    self.flatten = Flatten()

  def call(self, pair):
    target, context = pair
    word_emb = self.target_embedding(target)
    context_emb = self.context_embedding(context)
    dots = self.dots([context_emb, word_emb])
    return self.flatten(dots)

In [155]:
embedding_dim = 128
num_ns = 4
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [156]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
word2vec.fit(dataset, epochs=40, callbacks=[tensorboard_callback])

Epoch 1/40
397/397 [==============================] - 45s 111ms/step - loss: 1.6020 - accuracy: 0.2364
Epoch 2/40
397/397 [==============================] - 41s 104ms/step - loss: 1.5058 - accuracy: 0.4107
Epoch 3/40
397/397 [==============================] - 41s 105ms/step - loss: 1.3549 - accuracy: 0.5143
Epoch 4/40
397/397 [==============================] - 41s 104ms/step - loss: 1.1517 - accuracy: 0.6610
Epoch 5/40
397/397 [==============================] - 41s 103ms/step - loss: 0.9414 - accuracy: 0.7633
Epoch 6/40
397/397 [==============================] - 41s 104ms/step - loss: 0.7574 - accuracy: 0.8266
Epoch 7/40
397/397 [==============================] - 42s 106ms/step - loss: 0.6081 - accuracy: 0.8675
Epoch 8/40
397/397 [==============================] - 41s 103ms/step - loss: 0.4916 - accuracy: 0.8956
Epoch 9/40
397/397 [==============================] - 41s 104ms/step - loss: 0.4024 - accuracy: 0.9161
Epoch 10/40
397/397 [==============================] - 42s 106ms/step - l

In [156]:
%tensorboard --logdir logs

In [159]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()
import io

In [160]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [161]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>